In [48]:
import os
import requests
cred_url = os.environ["QCLOUD_CONTAINER_INSTANCE_CREDENTIALS_URL"]
r = requests.get(cred_url)
secretId = r.json()["TmpSecretId"]
secretKey = r.json()["TmpSecretKey"]
token = r.json()["Token"]

In [49]:
import os
from qcloud_cos import CosConfig
from qcloud_cos import CosS3Client
from ti.utils import get_temporary_secret_and_token

#### 指定本地文件路径，可根据需要修改。
local_file = "/home/tione/notebook/data/Round2_trainb.zip"

#### 用户的存储桶，修改为存放所需数据文件的存储桶，存储桶获取参考腾讯云对象存储
bucket="hc1-1258788535"

#### 用户的数据，修改为对应的数据文件路径，文件路径获取参考腾讯云对象存储
data_key="Round2_trainb.zip"

#### 获取用户临时密钥
secret_id, secret_key, token = get_temporary_secret_and_token()
config = CosConfig(Region=os.environ.get('REGION'), SecretId=secret_id, SecretKey=secret_key, Token=token, Scheme='https')
client = CosS3Client(config)

###  获取文件到本地
response = client.get_object(
    Bucket=bucket,
    Key=data_key,
)
response['Body'].get_stream_to_file(local_file)

In [1]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
import datetime, time 
from utils import distance, haversine, standard, pad_seq, geohash_encode
from scipy.stats import skew, kurtosis
from zipfile import ZipFile
from collections import Counter 
from sklearn.metrics import mean_squared_error as mse 
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# !pip install lightgbm
!free -m

              total        used        free      shared  buff/cache   available
Mem:          96166       21336       44895          18       29934       74666
Swap:             0           0           0


# 分析港口

In [26]:
port = pd.read_csv("data/port.csv")
print(port.shape)
port.head()
port['geo_hash4'] = port[["LONGITUDE", "LATITUDE"]].apply(lambda x: geohash_encode(x[0], x[1], precision=4), axis=1)

port['lonlat'] = port[["LONGITUDE", "LATITUDE"]].apply(lambda x: (x[0], x[1]), axis=1)
porthash2lonlat = dict(zip(port['geo_hash4'], port['lonlat']))
print(len(porthash2lonlat))

(2456, 10)
1226


In [1]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          96166       18361       62098          18       15706       77641
Swap:             0           0           0


In [5]:
%%time

myzip=ZipFile('data/Round2_trainb.zip')
f=myzip.open('Round2_trainb.csv')
train_df=pd.read_csv(f)
print(train_df.shape)
print(train_df.head())
f.close()
myzip.close()

train_df.drop_duplicates(['loadingOrder', 'label'], inplace=True)

del_cols = ['arrive_time_index', 'arrive_time', 'direction',  'begin_year', 'geo_hash3', 'selcte_index'] # 'or_last_index',
train_df.drop(del_cols, axis=1, inplace=True) 
print(train_df.columns)

columns = ['TRANSPORT_TRACE', 'begin_port_name', 'begin_port_position',
        'begin_port_position_hash3', 'begin_port_position_hash4',
        'begin_port_position_hash5', 'carrierName', 'end_port_name',
        'end_port_position', 'end_port_position_hash3',
        'end_port_position_hash4', 'end_port_position_hash5', 'loadingOrder',
        'vesselMMSI', 'test_index', 'geo_hash3', 'geo_hash5',
        'geo_hash4', 'speed', 'longitude',
        'latitude', 'timestamp', 'direction', 'label']
train_df.columns = columns

test_df = pd.read_csv("data/Round2_testb.csv")
print(train_df.shape, test_df.shape)

(223603, 30)
  TRANSPORT_TRACE               arrive_time  arrive_time_index  \
0     CNYTN-MYTPP  2020-03-27T18:32:53.000Z               2072   
1     CNYTN-MYTPP  2020-03-27T18:32:53.000Z               2072   
2     CNYTN-MYTPP  2020-03-27T18:32:53.000Z               2072   
3     CNSHK-MYTPP  2020-04-13T07:57:43.000Z                653   
4     CNSHK-MYTPP  2020-04-13T07:57:43.000Z                653   

  begin_port_name           begin_port_position begin_port_position_hash3  \
0           CNYTN            114.275347 22.5777                       ws1   
1           CNYTN            114.275347 22.5777                       ws1   
2           CNYTN            114.275347 22.5777                       ws1   
3           CNSHK  113.86305800000001 22.559462                       ws0   
4           CNSHK  113.86305800000001 22.559462                       ws0   

  begin_port_position_hash4 begin_port_position_hash5 carrierName  \
0                      ws12                     ws122     

In [6]:
train_df = train_df[train_df['label'] > 0].reset_index(drop=True)
print(train_df.shape)

(216358, 24)


In [7]:
test_df.head(1)

,loadingOrder,timestamp,direction,speed,TRANSPORT_TRACE,carrierName,vesselMMSI,longitude,latitude,geo_hash5,geo_hash4,geo_hash3,begin_port_name,end_port_name,begin_port_position,end_port_position,begin_port_position_hash4,begin_port_position_hash5,begin_port_position_hash3,end_port_position_hash4,end_port_position_hash5,end_port_position_hash3,right_index,ans
0,AB674675500650,2020-06-01T00:01:50.000Z 2020-06-01T00:04:48.0...,19400.0 19600.0 19900.0 21200.0 19700.0 16900....,0 0 0 1 3 4 5 4 3 2 2 2 2 2 2 2 2 2 2 2 1 1 2 ...,CNSHK-HKHKG-TWKHH-CNNBG-CNSHA-CNTAO-KRPUS-MXZL...,RWHZVZ,V2180946969,-79.91805 -79.918133 -79.918217 -79.9183829999...,9.257433 9.2571 9.25685 9.256467 9.2557 9.2545...,d1xht d1xht d1xht d1xht d1xht d1xht d1xht d1xh...,d1xh d1xh d1xh d1xh d1xh d1xh d1xh d1xh d1xh d...,d1x d1x d1x d1x d1x d1x d1x d1x d1x d1x d1x d1...,CNSHK,DOCAU,113.86305800000001 22.559462,-69.638318 18.425821,ws0b,ws0br,ws0,d7q2,d7q2z,d7q,-1,NaN


In [ ]:
%%time

def get_stats(values):   # 统计特征
    xlen = len(values)
    vmax = np.max(values)
    vmin = np.min(values)
    return [np.mean(values), 
            vmax, vmin, np.std(values), 
            # vmax-vmin,
            # np.sum(values), 
            # np.median(values), 
            xlen-np.count_nonzero(values), 
            values[0], values[-1],
            #np.count_nonzero(values)/xlen, 
            #np.percentile(values, 25), np.percentile(values, 75), 
            len(set(values)), 
            #len(set(values))/xlen,
            skew(values), 
            # kurtosis(values), 
            #sorted(Counter(values).items(), key=lambda x: x[1], reverse=True)[0][0]
           ]

def feature_engineering(df, is_train=True):
    numerical_fea = []
    categorical_fea = []
    if is_train:
        label = []
    for idx in tqdm(range(df.shape[0])):
        line = df.iloc[idx]

        timestamp = [datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z") for x in line['timestamp'].split(' ')]
        
        # 时间特征
        timestamp_hours = np.array([(x-timestamp[0]).total_seconds()/3600.0 for x in timestamp])  # 减去最初的时间
        timestamp_diff1 = np.diff(timestamp_hours, prepend=timestamp_hours[0])    # 时间一阶差分
        # print(timestamp_diff1[:20])
        xlen = len(timestamp_hours)   # 长度

        begin_port = [float(x) for x in line['begin_port_position'].split(' ')]
        end_port = [float(x) for x in line['end_port_position'].split(' ')]

        # 原始的四个特征
        lon = [float(x) for x in line['longitude'].split(' ')]  # 经度
        lat  = [float(x) for x in line['latitude'].split(' ')]  # 纬度
        speed = [float(x) for x in line['speed'].split(' ')]
        direction = [float(x) for x in line['direction'].split(' ')]
        lon_diff1 = np.diff(lon, prepend=lon[0])
        lat_diff1 = np.diff(lat, prepend=lat[0])
        speed_diff1 = np.diff(speed, prepend=speed[0])
        direction_diff1 = np.diff(direction, prepend=direction[0])

        # 类别特征
        carrierName = line['carrierName']
        vesselMMSI = line['vesselMMSI']
        begin_hash3, begin_hash4, begin_hash5 = line['begin_port_position_hash3'], line['begin_port_position_hash4'], line['begin_port_position_hash5']
        end_hash3, end_hash4, end_hash5 = line['end_port_position_hash3'], line['end_port_position_hash4'], line['end_port_position_hash5']
        first_hash3, first_hash4, first_hash5 = line['geo_hash3'].split(' ')[0], line['geo_hash4'].split(' ')[0], line['geo_hash5'].split(' ')[0]
        last_hash3, last_hash4, last_hash5 = line['geo_hash3'].split(' ')[-1], line['geo_hash4'].split(' ')[-1], line['geo_hash5'].split(' ')[-1]
        first_year, first_month, first_day = timestamp[0].year, timestamp[0].month, timestamp[0].day
        last_year, last_month, last_day = timestamp[-1].year, timestamp[-1].month, timestamp[-1].day

        # 距离特征
        begin_distance = [distance(begin_port[0], begin_port[1], lon[i], lat[i]) for i in range(xlen)]  # 与起始距离
        begin_dis_cumsum = np.cumsum(begin_distance)   # 累积距离
        begin_dis_diff1 = np.diff(begin_distance, prepend=begin_distance[0])   # 一阶差分距离

        end_distance = [distance(end_port[0], end_port[1], lon[i], lat[i]) for i in range(xlen)]    # 与终止距离
        end_dis_cumsum = np.cumsum(end_distance)    # 累积距离
        end_dis_diff1 = np.diff(end_distance, prepend=end_distance[0]) 

        begin_haversine = [haversine(begin_port[0], begin_port[1], lon[i], lat[i]) for i in range(xlen)] 
        begin_hav_cumsum = np.cumsum(begin_haversine)   # 累积距离
        begin_hav_diff1 = np.diff(begin_haversine, prepend=begin_haversine[0])   # 一阶差分距离

        end_haversine = [haversine(end_port[0], end_port[1], lon[i], lat[i]) for i in range(xlen)]
        end_hav_cumsum = np.cumsum(end_haversine)   # 累积距离
        end_hav_diff1 = np.diff(end_haversine, prepend=end_haversine[0])   # 一阶差分距离

        # 速度特征
        timestamp_diff1_tmp = timestamp_diff1.copy()
        timestamp_diff1_tmp[timestamp_diff1_tmp==0] = np.mean(timestamp_diff1_tmp)
        diff_speed = begin_hav_diff1 / (timestamp_diff1_tmp + 1e-8)    # 差分路程 / 差分时间 = 速度1
        
        timestamp_hours_tmp = timestamp_hours.copy()
        timestamp_hours_tmp[timestamp_hours_tmp == 0] = np.mean(timestamp_hours_tmp)
        cumsum_speed = begin_hav_cumsum / (timestamp_hours_tmp + 1e-8)  # 累积路程 / 累积时间 = 速度2

        # 简单数值特征
        speed_0_nums = (np.array(speed) == 0).sum()
        speed_0_rate = speed_0_nums / xlen 
        speed_0_time = timestamp_diff1[np.array(speed) == 0].sum()
        hash3_nunique = len(set(line['geo_hash3'].split(' ')))
        hash4_nunique = len(set(line['geo_hash4'].split(' ')))
        hash5_nunique = len(set(line['geo_hash5'].split(' ')))
        hash3_rate, hash4_rate, hash5_rate = hash3_nunique/xlen,  hash4_nunique/xlen, hash5_nunique/xlen 
        first_begin_dis, first_begin_hav = distance(begin_port[0], begin_port[1], lon[0], lat[0]), haversine(begin_port[0], begin_port[1], lon[0], lat[0])
        last_begin_dis, last_begin_hav = distance(begin_port[0], begin_port[1], lon[-1], lat[-1]), haversine(begin_port[0], begin_port[1], lon[-1], lat[-1])
        first_end_dis, first_end_hav = distance(end_port[0], end_port[1], lon[0], lat[0]), haversine(end_port[0], end_port[1], lon[0], lat[0])
        last_end_dis, last_end_hav = distance(end_port[0], end_port[1], lon[-1], lat[-1]), haversine(end_port[0], end_port[1], lon[-1], lat[-1])
        fbd_lbd, fbd_lbd_rate = last_begin_dis/first_begin_dis, last_begin_dis-first_begin_dis
        fbh_lbh, fbh_lbh_rate = last_begin_hav/first_begin_hav, last_begin_hav-first_begin_hav
        fed_led, fed_led_rate = first_end_dis/last_end_dis, first_end_dis-last_end_dis
        fed_leh, fed_leh_rate = first_end_hav/last_end_hav, first_end_hav-last_end_hav


        # 正式添加统计特征
        num_feas1 = get_stats(timestamp_hours) + get_stats(timestamp_diff1) + get_stats(lon) + get_stats(lat) + get_stats(speed) + get_stats(direction)
        num_feas2 = get_stats(begin_distance)+get_stats(begin_dis_cumsum)+get_stats(begin_dis_diff1)+get_stats(end_distance)+get_stats(end_dis_cumsum)+get_stats(end_dis_diff1)
        num_feas3 = get_stats(begin_haversine)+get_stats(begin_hav_cumsum)+get_stats(begin_hav_diff1)+get_stats(end_haversine)+get_stats(end_hav_cumsum)+get_stats(end_hav_diff1)
        num_feas4 = get_stats(diff_speed) + get_stats(cumsum_speed)
        num_feas5 = get_stats(lon_diff1) + get_stats(lat_diff1) + get_stats(speed_diff1) + get_stats(direction_diff1)
        num_feas6 = [xlen, speed_0_nums, speed_0_rate, speed_0_time, hash3_nunique, hash4_nunique, hash5_nunique, 
                     hash3_rate, hash4_rate, hash5_rate, first_begin_dis, first_begin_hav,  last_begin_dis, last_begin_hav,
                     first_end_dis, first_end_hav, last_end_dis, last_end_hav, fbd_lbd, fbd_lbd_rate, 
                     fbh_lbh, fbh_lbh_rate, fed_led, fed_led_rate, fed_leh, fed_leh_rate]
        # print(len(num_feas1+num_feas2+num_feas3))
        # break 

        # 添加类别型特征
        cat_feas1 = [carrierName, vesselMMSI, begin_hash3, begin_hash4, begin_hash5, 
                     end_hash3, end_hash4, end_hash5, first_hash3, first_hash4, first_hash5, 
                     last_hash3, last_hash4, last_hash5, first_year, first_month, first_day, 
                     last_year, last_month, last_day]
        
        numerical_fea.append(num_feas1 + num_feas2 + num_feas3 + num_feas4 + num_feas5 + num_feas6)
        categorical_fea.append(cat_feas1)
        if is_train:
            label.append(line['label'])
    
    numerical_fea = np.array(numerical_fea)
    categorical_fea = np.array(categorical_fea)
    
    # print(numerical_fea.shape)
    num_cols = ['num_{}'.format(str(i)) for i in range(numerical_fea.shape[1])]
    cat_cols = ['cat_{}'.format(str(i)) for i in range(categorical_fea.shape[1])]
    num_df = pd.DataFrame(numerical_fea)
    print(num_df.shape)
    cat_df = pd.DataFrame(categorical_fea)
    print(cat_df.shape)
    fea_df = pd.concat([num_df, cat_df], axis=1)
    
    fea_df.columns = num_cols + cat_cols
    if is_train:
        fea_df['label'] = label
    print(fea_df.shape)
    return fea_df

test_fea = feature_engineering(test_df, is_train=False)
train_fea = feature_engineering(train_df)

  0%|          | 5/216358 [00:00<1:23:57, 42.95it/s]

(239, 242)
(239, 20)
(239, 262)


 10%|▉         | 21609/216358 [20:50<2:54:21, 18.62it/s] 

In [10]:
train_fea.to_pickle('data/train_fea_0809.pkl')
test_fea.to_pickle('data/test_fea_0809.pkl')

In [14]:
train_fea = pd.read_pickle("data/train_fea_0808.pkl")
test_fea = pd.read_pickle("data/test_fea_0808.pkl")

In [9]:
print(train_fea.shape, test_fea.shape)

(216358, 263) (239, 262)


In [11]:
test_fea.head()

,num_0,num_1,num_2,num_3,num_4,num_5,num_6,num_7,num_8,num_9,num_10,num_11,num_12,num_13,num_14,num_15,num_16,num_17,num_18,num_19,num_20,num_21,num_22,num_23,num_24,num_25,num_26,num_27,num_28,num_29,num_30,num_31,num_32,num_33,num_34,num_35,num_36,num_37,num_38,num_39,num_40,num_41,num_42,num_43,num_44,num_45,num_46,num_47,num_48,num_49,num_50,num_51,num_52,num_53,num_54,num_55,num_56,num_57,num_58,num_59,num_60,num_61,num_62,num_63,num_64,num_65,num_66,num_67,num_68,num_69,num_70,num_71,num_72,num_73,num_74,num_75,num_76,num_77,num_78,num_79,num_80,num_81,num_82,num_83,num_84,num_85,num_86,num_87,num_88,num_89,num_90,num_91,num_92,num_93,num_94,num_95,num_96,num_97,num_98,num_99,num_100,num_101,num_102,num_103,num_104,num_105,num_106,num_107,num_108,num_109,num_110,num_111,num_112,num_113,num_114,num_115,num_116,num_117,num_118,num_119,num_120,num_121,num_122,num_123,num_124,num_125,num_126,num_127,num_128,num_129,num_130,num_131,num_132,num_133,num_134,num_135,num_136,num_137,num_138,num_139,num_140,num_141,num_142,num_143,num_144,num_145,num_146,num_147,num_148,num_149,num_150,num_151,num_152,num_153,num_154,num_155,num_156,num_157,num_158,num_159,num_160,num_161,num_162,num_163,num_164,num_165,num_166,num_167,num_168,num_169,num_170,num_171,num_172,num_173,num_174,num_175,num_176,num_177,num_178,num_179,num_180,num_181,num_182,num_183,num_184,num_185,num_186,num_187,num_188,num_189,num_190,num_191,num_192,num_193,num_194,num_195,num_196,num_197,num_198,num_199,num_200,num_201,num_202,num_203,num_204,num_205,num_206,num_207,num_208,num_209,num_210,num_211,num_212,num_213,num_214,num_215,num_216,num_217,num_218,num_219,num_220,num_221,num_222,num_223,num_224,num_225,num_226,num_227,num_228,num_229,num_230,num_231,num_232,num_233,num_234,num_235,num_236,num_237,num_238,num_239,num_240,num_241,cat_0,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19
0,10.722610,23.934722,0.0,7.181429,1.0,0.0,23.934722,440.0,0.216247,0.054397,0.105000,0.0,0.025264,1.0,0.0,0.050278,135.0,0.718651,-79.893851,-79.885233,-79.919283,0.012923,0.0,-79.918050,-79.885250,143.0,-0.999897,9.348437,9.423383,9.252117,0.042348,0.0,9.257433,9.369717,150.0,-1.291235,1.765909,27.0,0.0,4.553942,332.0,0.0,0.0,25.0,3.201669,18094.318182,35900.0,0.0,10832.801178,8.0,19400.0,8600.0,144.0,0.104324,194.206778,194.238007,194.196732,0.015226,0.0,194.237128,194.196746,284.0,0.995461,42824.139686,85450.982434,194.237128,24667.116224,0.0,194.237128,85450.982434,440.0,-0.000075,-0.000092,0.000535,-0.006600,0.000514,17.0,0.0,-0.000125,421.0,-8.188465,13.695821,13.777164,13.655107,0.036076,0.0,13.774332,13.675256,284.0,1.275499,3023.387763,6026.161254,13.774332,1738.478208,0.0,13.774332,6026.161254,440.0,-0.002031,-0.000225,0.003715,-0.007548,0.000977,17.0,0.0,-0.000165,421.0,-2.745482,16163.510623,16172.501792,16155.100026,3.936102,0.0,16171.814312,16161.693447,284.0,1.222800,3.564384e+06,7.111945e+06,16171.814312,2.052911e+06,0.0,16171.814312,7.111945e+06,440.0,-0.000141,-0.023002,0.663457,-0.904947,0.150719,17.0,0.0,-0.007047,421.0,-1.068595,1497.031756,1506.255694,1492.270906,4.088396,0.0,1505.930056,1494.716519,284.0,1.280985,330486.012899,658693.972450,1505.930056,190020.826964,0.0,1505.930056,658693.972450,440.0,-0.002087,-0.025485,0.427914,-0.843205,0.111882,17.0,0.0,-0.018266,421.0,-2.635384,-0.686151,19.903695,-25.123987,4.504278,17.0,0.0,-0.140166,424.0,-0.972426,368053.796788,654141.290174,1508.197498,60733.324033,0.0,1508.197498,297139.219126,440.0,-0.033306,0.000075,0.006533,-0.000733,0.000518,44.0,0.0,0.000117,118.0,7.998333,0.000255,0.008900,-0.005433,0.001428,50.0,0.0,0.000117,146.0,1.546439,0.000000,4.0,-3.0,0.625227,370.0,0.0,0.0,8.0,0.278968,-24.545455,35900.0,-35900.0,7847.656701,230.0,0.0,-1900.0,121.0,-0.236285,440.0,332.0,0.754545,20.261667,1.0,2.0,6.0,0.002273,0.004545,0.013636,194.237128,16171.814312,194.196746,16161.693447,13.774332,1505.930056,13.675256,1494.716519,0.999792,

# 读取特征文件

In [4]:
# train_weight = train_fea['num_336'].values

In [17]:
print(train_fea.shape, test_fea.shape)
train_fea.head()

(199770, 263) (200, 262)


,num_0,num_1,num_2,num_3,num_4,num_5,num_6,num_7,num_8,num_9,num_10,num_11,num_12,num_13,num_14,num_15,num_16,num_17,num_18,num_19,num_20,num_21,num_22,num_23,num_24,num_25,num_26,num_27,num_28,num_29,num_30,num_31,num_32,num_33,num_34,num_35,num_36,num_37,num_38,num_39,num_40,num_41,num_42,num_43,num_44,num_45,num_46,num_47,num_48,num_49,num_50,num_51,num_52,num_53,num_54,num_55,num_56,num_57,num_58,num_59,num_60,num_61,num_62,num_63,num_64,num_65,num_66,num_67,num_68,num_69,num_70,num_71,num_72,num_73,num_74,num_75,num_76,num_77,num_78,num_79,num_80,num_81,num_82,num_83,num_84,num_85,num_86,num_87,num_88,num_89,num_90,num_91,num_92,num_93,num_94,num_95,num_96,num_97,num_98,num_99,num_100,num_101,num_102,num_103,num_104,num_105,num_106,num_107,num_108,num_109,num_110,num_111,num_112,num_113,num_114,num_115,num_116,num_117,num_118,num_119,num_120,num_121,num_122,num_123,num_124,num_125,num_126,num_127,num_128,num_129,num_130,num_131,num_132,num_133,num_134,num_135,num_136,num_137,num_138,num_139,num_140,num_141,num_142,num_143,num_144,num_145,num_146,num_147,num_148,num_149,num_150,num_151,num_152,num_153,num_154,num_155,num_156,num_157,num_158,num_159,num_160,num_161,num_162,num_163,num_164,num_165,num_166,num_167,num_168,num_169,num_170,num_171,num_172,num_173,num_174,num_175,num_176,num_177,num_178,num_179,num_180,num_181,num_182,num_183,num_184,num_185,num_186,num_187,num_188,num_189,num_190,num_191,num_192,num_193,num_194,num_195,num_196,num_197,num_198,num_199,num_200,num_201,num_202,num_203,num_204,num_205,num_206,num_207,num_208,num_209,num_210,num_211,num_212,num_213,num_214,num_215,num_216,num_217,num_218,num_219,num_220,num_221,num_222,num_223,num_224,num_225,num_226,num_227,num_228,num_229,num_230,num_231,num_232,num_233,num_234,num_235,num_236,num_237,num_238,num_239,num_240,num_241,cat_0,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19,label
0,206.538927,318.100556,0.0,93.035520,1.0,0.0,318.100556,1847.0,-0.733368,0.172226,2.933333,0.0,0.321751,1.0,0.0,0.001667,833.0,2.866693,-51.166123,179.952767,-179.983550,143.080260,0.0,138.928747,-127.299227,1844.0,0.784626,-33.758206,2.374827,-46.395387,14.911222,0.0,2.374827,-46.395387,1826.0,0.980042,31.603682,37.0,26.0,2.332174,0.0,34.0,27.0,12.0,-0.475608,11305.793178,19800.0,7700.0,1398.682188,0.0,13000.0,9600.0,60.0,0.398637,203.189281,298.723384,31.879491,99.450596,0.0,31.879491,251.054388,1844.0,-0.784826,147665.488799,375290.602166,31.879491,121624.648293,0.0,31.879491,375290.602166,1847.0,0.323246,0.118665,214.743730,-0.794415,4.996663,4.0,0.0,-0.000440,1844.0,42.907552,125.630606,251.617210,57.238339,73.014296,0.0,213.527561,57.238339,1844.0,0.752005,149502.913337,232039.730195,213.527561,64987.140676,0.0,213.527561,232039.730195,1847.0,-0.755377,-0.084618,0.745881,-143.228561,3.333266,4.0,0.0,-0.000448,1844.0,-42.878719,10404.613694,13957.134113,3482.118511,3077.853066,0.0,3482.118511,13957.134113,1844.0,-0.756182,8.058294e+06,1.921732e+07,3482.118511,5.818938e+06,0.0,3482.118511,1.921732e+07,1847.0,0.296303,5.671367,102.216582,-0.149663,10.692107,4.0,0.0,0.036097,1844.0,2.949795,8444.361325,15351.176915,4878.157334,3086.034476,0.0,15351.176915,4878.157334,1844.0,0.749771,9.362943e+06,1.559674e+07,15351.176915,4.334593e+06,0.0,15351.176915,1.559674e+07,1847.0,-0.453485,-5.670287,0.150478,-99.930123,10.672759,4.0,0.0,-0.037197,1844.0,-2.920544,32.041307,213.592486,-193.256755,9.596072,4.0,0.0,21.658313,1844.0,-1.881558,35426.031477,2.089297e+06,16.859381,55223.705220,0.0,16.859381,60412.725335,1847.0,29.434367,-0.144141,0.866950,-359.936317,8.374620,4.0,0.0,0.000480,1291.0,-42.933134,-0.026405,0.009400,-0.490184,0.054604,20.0,0.0,-0.000080,1096.0,-3.262603,-0.003790,2.0,-3.0,0.442083,1512.0,0.0,0.0,6.0,-0.243737,-1.840823,7700.0,-7400.0,314.901297,716.0,0.0,100.0,26.0,0.638010,1847.0,0.0,0.000000,0.000000,97.0,378.0,694.0,0.052518,0.204656,0.375744,31.879491,3482.118511,251.054388,13957.13

In [24]:
test_fea.head()

,num_0,num_1,num_2,num_3,num_4,num_5,num_6,num_7,num_8,num_9,num_10,num_11,num_12,num_13,num_14,num_15,num_16,num_17,num_18,num_19,num_20,num_21,num_22,num_23,num_24,num_25,num_26,num_27,num_28,num_29,num_30,num_31,num_32,num_33,num_34,num_35,num_36,num_37,num_38,num_39,num_40,num_41,num_42,num_43,num_44,num_45,num_46,num_47,num_48,num_49,num_50,num_51,num_52,num_53,num_54,num_55,num_56,num_57,num_58,num_59,num_60,num_61,num_62,num_63,num_64,num_65,num_66,num_67,num_68,num_69,num_70,num_71,num_72,num_73,num_74,num_75,num_76,num_77,num_78,num_79,num_80,num_81,num_82,num_83,num_84,num_85,num_86,num_87,num_88,num_89,num_90,num_91,num_92,num_93,num_94,num_95,num_96,num_97,num_98,num_99,num_100,num_101,num_102,num_103,num_104,num_105,num_106,num_107,num_108,num_109,num_110,num_111,num_112,num_113,num_114,num_115,num_116,num_117,num_118,num_119,num_120,num_121,num_122,num_123,num_124,num_125,num_126,num_127,num_128,num_129,num_130,num_131,num_132,num_133,num_134,num_135,num_136,num_137,num_138,num_139,num_140,num_141,num_142,num_143,num_144,num_145,num_146,num_147,num_148,num_149,num_150,num_151,num_152,num_153,num_154,num_155,num_156,num_157,num_158,num_159,num_160,num_161,num_162,num_163,num_164,num_165,num_166,num_167,num_168,num_169,num_170,num_171,num_172,num_173,num_174,num_175,num_176,num_177,num_178,num_179,num_180,num_181,num_182,num_183,num_184,num_185,num_186,num_187,num_188,num_189,num_190,num_191,num_192,num_193,num_194,num_195,num_196,num_197,num_198,num_199,num_200,num_201,num_202,num_203,num_204,num_205,num_206,num_207,num_208,num_209,num_210,num_211,num_212,num_213,num_214,num_215,num_216,num_217,num_218,num_219,num_220,num_221,num_222,num_223,num_224,num_225,num_226,num_227,num_228,num_229,num_230,num_231,num_232,num_233,num_234,num_235,num_236,num_237,num_238,num_239,num_240,num_241,num_242,num_243,num_244,num_245,num_246,num_247,num_248,num_249,num_250,num_251,num_252,num_253,num_254,num_255,num_256,num_257,num_258,num_259,num_260,num_261,num_262,num_263,num_264,num_265,num_266,num_267,num_268,num_269,num_270,num_271,num_272,num_273,num_274,num_275,num_276,num_277,num_278,num_279,num_280,num_281,num_282,num_283,num_284,num_285,num_286,num_287,num_288,num_289,num_290,num_291,num_292,num_293,num_294,num_295,num_296,num_297,num_298,num_299,num_300,num_301,num_302,num_303,num_304,num_305,num_306,num_307,num_308,num_309,num_310,num_311,num_312,num_313,num_314,num_315,num_316,num_317,num_318,num_319,num_320,num_321,num_322,num_323,num_324,num_325,num_326,num_327,num_328,num_329,num_330,num_331,num_332,num_333,num_334,num_335,num_336,num_337,num_338,num_339,num_340,num_341,num_342,num_343,num_344,num_345,num_346,num_347,num_348,num_349,num_350,num_351,num_352,num_353,num_354,num_355,num_356,num_357,num_358,num_359,num_360,num_361,cat_0,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19
0,124.754499,251.402778,0.0,82.613136,251.402778,41668.002778,141.875000,333.0,0.997006,334.0,1.0,-0.333507,-1.216325,0.0,0.752703,21.722222,0.0,1.785728,21.722222,251.402778,0.187500,333.0,0.997006,248.0,0.742515,7.410639,72.379045,0.016667,138.465947,162.527333,113.895823,16.560653,48.631510,46247.626273,142.308720,334.0,1.0,333.0,0.997006,-0.368396,-1.257847,114.070615,1.960866,22.383463,-21.223300,14.342174,43.606763,654.929202,-0.922960,334.0,1.0,331.0,0.991018,0.231913,-1.259498,22.345372,26.089820,38.0,0.0,8.341747,38.0,8714.0,29.0,331.0,0.991018,33.0,0.098802,-1.710247,1.787737,30.0,13163.949102,31650.0,-1.0,2956.961801,31651.0,4396759.0,13350.0,334.0,1.0,231.0,0.691617,1.268326,12.547578,13350.0,32.097839,65.460995,0.179023,21.892607,65.281972,10720.678084,36.886038,334.0,1.0,333.0,0.997006,-0.312193,-1.261837,0.290722,3311.967506,10720.678084,0.179023,3334.005146,10720.499061,1.106197e+06,2284.175553,334.0,1.0,334.0,1.0,0.668160,-0.885724,0.179023,0.195455,6.724819,-0.004340,0.502955,6.729159,65.281972,0.046133,332.0,0.994012,333.0,0.997006,8.499139,94.5411

In [12]:
use_cols = []
# imp = pd.read_pickle("imp_2.pkl")
# del_cols = imp['feat'].values.tolist()[-30:]

for col in ['num_{}'.format(str(i)) for i in range(242)]:
    if np.std(train_fea[col].values) > 0.01:
        use_cols.append(col)
print(len(use_cols))

218


In [13]:
# label_encoder
from sklearn.preprocessing import LabelEncoder

test_fea['label'] = -1e8
data_fea = pd.concat([train_fea, test_fea])
data_fea = data_fea.reset_index(drop=True)

for col in ['cat_{}'.format(str(i)) for i in range(20)]:
    print(col)
    le = LabelEncoder()
    data_fea[col] = le.fit_transform(data_fea[col])
    data_fea[col] = data_fea[col].astype('category')

train_feas = data_fea[data_fea['label'] != -1e8].reset_index(drop=True)
test_feas = data_fea[data_fea['label'] == -1e8].reset_index(drop=True)
del test_feas['label']
print(train_feas.shape, test_feas.shape)

all_feas = use_cols + ['cat_{}'.format(str(i)) for i in range(20)]
cat_feas = ['cat_{}'.format(str(i)) for i in range(20)]
print(len(all_feas), len(cat_feas))

cat_0
cat_1
cat_2
cat_3
cat_4
cat_5
cat_6
cat_7
cat_8
cat_9
cat_10
cat_11
cat_12
cat_13
cat_14
cat_15
cat_16
cat_17
cat_18
cat_19
(216358, 263) (239, 262)
238 20


In [13]:
imp = pd.read_pickle("result/fea1.pkl")

imp['gain'] = imp[[f for f in imp.columns if 'gain' in f]].sum(axis=1)/5
imp['split'] = imp[[f for f in imp.columns if 'split' in f]].sum(axis=1)/5

imp = imp[['feat', 'gain', 'split']]
imp = imp.sort_values(by=['gain'], ascending=False)
imp = imp.reset_index(drop=True)
# imp.to_pickle("imp_2.pkl")
# imp

all_feas = imp['feat'].values[:-30].tolist()
print(len(all_feas))

182


In [20]:
print(len(all_feas))

238


In [14]:
# Log
file_name = datetime.date.today().strftime('%m%d')+"_{}.log".format("lgb_round2")
def write_log(w):
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}\n".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info)
        f.write("-"*80+"\n")

In [24]:
# del label

In [23]:
# label = train_feas['label'].values
# print(label[100:120])
# label[label < 0] = 0
# print(label[100:120])

In [ ]:
%%time

def mse_score_eval(preds, valid):
    labels = valid.get_label()
    #weight = valid.data['num_336'].values
    scores = mse(y_true=labels, y_pred=preds)
    return 'mse_score', scores, False

def build_model(train, test, label, seed=2020, is_shuffle=True):
    imp = pd.DataFrame()  # 特征重要性
    imp['feat'] = all_feas

    train_pred = np.zeros((train.shape[0], ))
    test_pred = np.zeros((test.shape[0], ))
    n_splits = 10
    # Kfold
#     fold = GroupKFold(n_splits=n_splits)
    fold = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=seed)
#     groups = train_df['loadingOrder'].values
#     kf_way = fold.split(train, groups=groups)
    kf_way = fold.split(train)
    # params
    params = {
        'learning_rate': 0.02,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 128,
        'feature_fraction': 0.90,
        'bagging_fraction': 0.90,
        'bagging_freq': 5,
        'seed': 8,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 45,
        'verbose': 1,
    }
    # train
    for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
        write_log("fold {}".format(n_fold))
        train_x, train_y = train.iloc[train_idx], label[train_idx]
        valid_x, valid_y = train.iloc[valid_idx], label[valid_idx]
        # 数据加载
        n_train = lgb.Dataset(train_x, label=train_y, 
                             # free_raw_data=False
                             )
        n_valid = lgb.Dataset(valid_x, label=valid_y, 
                              #free_raw_data=False
                             )

        clf = lgb.train(
            params=params,
            train_set=n_train,
            categorical_feature=cat_feas,
            num_boost_round=15000,
            valid_sets=[n_train, n_valid],
            early_stopping_rounds=50,
            verbose_eval=400,
            feval=mse_score_eval
        )
        clf.save_model("data/lgb_save_model/lgb_model_fold_{}.txt".format(n_fold))
        train_pred[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        
        write_log("val_mse = {}".format(mse(valid_y, train_pred[valid_idx])))
        test_pred += clf.predict(test, num_iteration=clf.best_iteration)/fold.n_splits

        imp['gain' + str(n_fold + 1)] = clf.feature_importance(importance_type='gain')
        imp['split' + str(n_fold + 1)] = clf.feature_importance(importance_type='split')
        
    write_log("train mse: {}".format(mse(label, train_pred)))
    result = pd.DataFrame({
        'loadingOrder': test_df['loadingOrder'], 
        'label': test_pred,
    })
    return result, imp

result, imp = build_model(train_feas[all_feas], test_feas[all_feas], train_feas['label'].values, is_shuffle=True)

result.to_pickle("result/result1.pkl")
imp.to_pickle("result/fea1.pkl")
# 373.12617949583523  381.9896222463546   386.2948549012397

13:07:38 : fold 1



/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[400]	training's l2: 186.504	training's mse_score: 186.504	valid_1's l2: 482.374	valid_1's mse_score: 482.374
[800]	training's l2: 80.2265	training's mse_score: 80.2265	valid_1's l2: 387.968	valid_1's mse_score: 387.968
[1200]	training's l2: 46.4664	training's mse_score: 46.4664	valid_1's l2: 359.216	valid_1's mse_score: 359.216
[1600]	training's l2: 30.183	training's mse_score: 30.183	valid_1's l2: 343.977	valid_1's mse_score: 343.977
[2000]	training's l2: 20.8254	training's mse_score: 20.8254	valid_1's l2: 335.03	valid_1's mse_score: 335.03
[2400]	training's l2: 14.9876	training's mse_score: 14.9876	valid_1's l2: 329.248	valid_1's mse_score: 329.248
[2800]	training's l2: 11.0898	training's mse_score: 11.0898	valid_1's l2: 325.369	valid_1's mse_score: 325.369
[3200]	training's l2: 8.54132	training's mse_score: 8.54132	valid_1's l2: 322.925	valid_1's mse_score: 322.925
[3600]	training's l2: 6.73894	training's mse_score: 6.738

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[400]	training's l2: 182.335	training's mse_score: 182.335	valid_1's l2: 517.562	valid_1's mse_score: 517.562
[800]	training's l2: 78.5078	training's mse_score: 78.5078	valid_1's l2: 438.063	valid_1's mse_score: 438.063
[1200]	training's l2: 45.4944	training's mse_score: 45.4944	valid_1's l2: 415.71	valid_1's mse_score: 415.71
[1600]	training's l2: 29.5036	training's mse_score: 29.5036	valid_1's l2: 403.562	valid_1's mse_score: 403.562
[2000]	training's l2: 20.4228	training's mse_score: 20.4228	valid_1's l2: 396.499	valid_1's mse_score: 396.499
[2400]	training's l2: 14.881	training's mse_score: 14.881	valid_1's l2: 392.055	valid_1's mse_score: 392.055
[2800]	training's l2: 11.1638	training's mse_score: 11.1638	valid_1's l2: 389	valid_1's mse_score: 389
[3200]	training's l2: 8.56198	training's mse_score: 8.56198	valid_1's l2: 386.992	valid_1's mse_score: 386.992
[3600]	training's l2: 6.69854	training's mse_score: 6.69854	valid

...

Training until validation scores don't improve for 50 rounds
[400]	training's l2: 185.786	training's mse_score: 185.786	valid_1's l2: 467.579	valid_1's mse_score: 467.579
[800]	training's l2: 80.366	training's mse_score: 80.366	valid_1's l2: 379.917	valid_1's mse_score: 379.917
[1200]	training's l2: 46.6884	training's mse_score: 46.6884	valid_1's l2: 352.292	valid_1's mse_score: 352.292
[1600]	training's l2: 30.1513	training's mse_score: 30.1513	valid_1's l2: 338.311	valid_1's mse_score: 338.311
[2000]	training's l2: 20.7509	training's mse_score: 20.7509	valid_1's l2: 329.986	valid_1's mse_score: 329.986
[2400]	training's l2: 14.8302	training's mse_score: 14.8302	valid_1's l2: 324.285	valid_1's mse_score: 324.285
[2800]	training's l2: 11.1327	training's mse_score: 11.1327	valid_1's l2: 320.809	valid_1's mse_score: 320.809
[3200]	training's l2: 8.52407	training's mse_score: 8.52407	valid_1's l2: 318.217	valid_1's mse_score: 318.217
[3600]	training's l2: 6.7057	training's mse_score: 6.70

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[400]	training's l2: 182.3	training's mse_score: 182.3	valid_1's l2: 483.787	valid_1's mse_score: 483.787
[800]	training's l2: 79.4612	training's mse_score: 79.4612	valid_1's l2: 403.701	valid_1's mse_score: 403.701
[1200]	training's l2: 46.4746	training's mse_score: 46.4746	valid_1's l2: 376.389	valid_1's mse_score: 376.389
[1600]	training's l2: 30.5377	training's mse_score: 30.5377	valid_1's l2: 363.342	valid_1's mse_score: 363.342
[2000]	training's l2: 21.2569	training's mse_score: 21.2569	valid_1's l2: 355.416	valid_1's mse_score: 355.416
[2400]	training's l2: 15.3021	training's mse_score: 15.3021	valid_1's l2: 350.309	valid_1's mse_score: 350.309
[2800]	training's l2: 11.3443	training's mse_score: 11.3443	valid_1's l2: 346.928	valid_1's mse_score: 346.928
[3200]	training's l2: 8.64707	training's mse_score: 8.64707	valid_1's l2: 344.349	valid_1's mse_score: 344.349
[3600]	training's l2: 6.74801	training's mse_score: 6.748

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[400]	training's l2: 185.685	training's mse_score: 185.685	valid_1's l2: 432.589	valid_1's mse_score: 432.589
[800]	training's l2: 80.2699	training's mse_score: 80.2699	valid_1's l2: 358.122	valid_1's mse_score: 358.122
[1200]	training's l2: 46.0735	training's mse_score: 46.0735	valid_1's l2: 332.257	valid_1's mse_score: 332.257
[1600]	training's l2: 30.0127	training's mse_score: 30.0127	valid_1's l2: 319.781	valid_1's mse_score: 319.781


In [24]:
result = pd.read_pickle("result/result1.pkl")

In [17]:
print(result.shape)
result

(239, 2)


,loadingOrder,label
0,AB674675500650,125.770627
1,AC710522369160,138.417223
2,AD584528667006,56.507135
3,AD852352572660,321.337189
4,AE378244933121,135.485514
5,AE563102255135,576.043202
6,AI823168582586,531.357053
7,AJ216944869611,494.341648
8,AM438554857021,482.605668
9,AO528516733940,90.210673


In [97]:
result

,loadingOrder,label
0,AC437723355280,416.230213
1,AR701843002140,89.971870
2,AT407433613767,315.436076
3,AV639724236766,260.494004
4,BA498843975994,65.139755
5,BB470439135271,705.406683
6,BE790761427541,376.345920
7,BE898272362291,524.949900
8,BE929753510083,499.783716
9,BK898009012594,444.691697


In [26]:
result.loc[result['loadingOrder'] == 'ZU783006492351', 'label'] = 215.39
# result.loc[result['loadingOrder'] == 'HB956271385453', 'label'] = 623

# 根据保存的模型直接预测

In [8]:
def mse_score_eval(preds, valid):
    labels = valid.get_label()
    scores = mse(y_true=labels, y_pred=preds)
    return 'mse_score', scores, False

def build_model(train, test, label, seed=2020, is_shuffle=True):
    imp = pd.DataFrame()  # 特征重要性
    imp['feat'] = all_feas

    train_pred = np.zeros((train.shape[0], ))
    test_pred = np.zeros((test.shape[0], ))
    n_splits = 10
    # Kfold
    # fold = GroupKFold(n_splits=n_splits)
    fold = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=seed)
    # groups = train_df['loadingOrder'].values
    kf_way = fold.split(train)
    # params
    params = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 128,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.75,
        'bagging_freq': 5,
        'seed': 8,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 24,
        'verbose': 1,
    }
    # train
    for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
        print("fold ", n_fold)
        train_x, train_y = train.iloc[train_idx], label[train_idx]
        valid_x, valid_y = train.iloc[valid_idx], label[valid_idx]
        # 数据加载
        n_train = lgb.Dataset(train_x, label=train_y)
        n_valid = lgb.Dataset(valid_x, label=valid_y)

#         clf = lgb.train(
#             params=params,
#             train_set=n_train,
#             categorical_feature=cat_feas,
#             num_boost_round=10000,
#             valid_sets=[n_train, n_valid],
#             early_stopping_rounds=50,
#             verbose_eval=100,
#             feval=mse_score_eval
#         )
#         clf.save_model("data/lgb_model_fold_{}.txt".format(n_fold))
        clf = lgb.Booster(model_file="data/lgb_save_model/lgb_model_fold_{}.txt".format(n_fold))
        train_pred[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        print("val_mse = {}".format(mse(valid_y, train_pred[valid_idx])))
        test_pred += clf.predict(test, num_iteration=clf.best_iteration)/fold.n_splits

        imp['gain' + str(n_fold + 1)] = clf.feature_importance(importance_type='gain')
        imp['split' + str(n_fold + 1)] = clf.feature_importance(importance_type='split')

    print("train mse: ", mse(label, train_pred))
    result = pd.DataFrame({
        'loadingOrder': test_df['loadingOrder'], 
        'label': test_pred,
    })
    return result, imp

result, imp = build_model(train_feas[all_feas], test_feas[all_feas], train_feas['label'].values, is_shuffle=True)

fold  1
val_mse = 896.7173560462967
fold  2
val_mse = 517.5055837664042
fold  3
val_mse = 690.3456844017744
fold  4
val_mse = 680.1628377551775
fold  5
val_mse = 654.7095421786684
fold  6
val_mse = 831.7647606373238
fold  7
val_mse = 1160.7401751905188
fold  8
val_mse = 771.1870089634364
fold  9
val_mse = 871.6475767371815
fold  10
val_mse = 612.6248600797542
train mse:  768.7405385756534


In [123]:
imp['gain'] = imp[[f for f in imp.columns if 'gain' in f]].sum(axis=1)/5
imp['split'] = imp[[f for f in imp.columns if 'split' in f]].sum(axis=1)/5

imp = imp[['feat', 'gain', 'split']]
imp = imp.sort_values(by=['gain'], ascending=False)
imp = imp.reset_index(drop=True)
# imp.to_pickle("imp_2.pkl")
imp

,feat,gain,split
0,num_353,3.560980e+10,787.0
1,num_212,9.941364e+09,1136.6
2,num_352,5.077498e+09,725.8
3,cat_5,1.816445e+09,3354.2
4,num_128,1.718637e+09,1003.0
5,cat_6,9.524393e+08,4173.0
6,cat_1,9.034291e+08,28995.2
7,cat_7,3.546282e+08,7499.2
8,cat_11,2.455927e+08,15651.4
9,num_127,1.456713e+08,734.2


In [27]:
train_feas['label'].describe([0.01, 0.08])

count    130220.000000
mean        198.930673
std         198.848518
min       -1109.135556
1%          -44.315000
8%            0.544311
50%         121.855833
max        1388.678333
Name: label, dtype: float64

In [31]:
test_df.head(1)

,loadingOrder,timestamp,direction,speed,TRANSPORT_TRACE,carrierName,vesselMMSI,longitude,latitude,geo_hash5,geo_hash4,geo_hash3,begin_port_name,end_port_name,begin_port_position,end_port_position,begin_port_position_hash4,begin_port_position_hash5,begin_port_position_hash3,end_port_position_hash4,end_port_position_hash5,end_port_position_hash3,right_index,ans
0,AC437723355280,2019-06-23T02:53:18.000Z 2019-06-23T03:01:28.0...,16060 11910 8620 9110 9010 9010 9030 8960 9080...,26 27 23 23 23 23 23 23 23 20 17 17 18 19 20 2...,CNSHK-CLVAP,JCMFTA,G9916514058,113.895823 113.917878 113.97771200000001 113.9...,22.383463 22.360128 22.345253 22.34531 22.3453...,webzr wecp0 wecp1 wecp1 wecp1 wecp1 wecp1 wecp...,webz wecp wecp wecp wecp wecp wecp wecp wecp w...,web wec wec wec wec wec wec wec wec wec wec we...,CNSHK,CLVAP,113.86305800000001 22.559462,-71.642993 -33.030843,ws0b,ws0br,ws0,66jh,66jh1,66j,-1,NaN


In [66]:
result = pd.read_pickle("result/result1.pkl")

In [67]:
test_df = pd.read_csv("data/Round2_test_802.csv")
test_df2 = pd.read_csv("data/Round2_test_717.csv")

In [68]:
test_df[test_df['loadingOrder']=="BR663094574600"]

,loadingOrder,timestamp,direction,speed,TRANSPORT_TRACE,carrierName,vesselMMSI,longitude,latitude,geo_hash5,geo_hash4,geo_hash3,begin_port_name,end_port_name,begin_port_position,end_port_position,begin_port_position_hash4,begin_port_position_hash5,begin_port_position_hash3,end_port_position_hash4,end_port_position_hash5,end_port_position_hash3,right_index,ans
13,BR663094574600,2020-06-01T00:02:56.000Z,33380,3,CNSHK-MYTPP-MUPLU-ZADUR,RWHZVZ,C5703488476,31.238667,-29.73064,kdwny,kdwn,kdw,CNSHK,ZADUR,113.86305800000001 22.559462,31.050079999999998 -29.868304,ws0b,ws0br,ws0,kdwn,kdwn4,kdw,0,2020-06-01T00:07:59.000Z


In [69]:
test_df2[test_df2['loadingOrder']=="BR663094574600"] 

,loadingOrder,timestamp,direction,speed,TRANSPORT_TRACE,carrierName,vesselMMSI,longitude,latitude,geo_hash5,geo_hash4,geo_hash3,begin_port_name,end_port_name,begin_port_position,end_port_position,begin_port_position_hash4,begin_port_position_hash5,begin_port_position_hash3,end_port_position_hash4,end_port_position_hash5,end_port_position_hash3
14,BR663094574600,2020-06-01T00:02:56.000Z 2020-06-01T00:07:59.0...,33380 14950 15090 15390 15620 15070 34030 3446...,3 2 0 3 0 7 1 2 1 2 1 2 3 0 1 2 6 0 4 1 0 4 4 ...,CNSHK-MYTPP-MUPLU-ZADUR,RWHZVZ,C5703488476,31.238667 31.23872 31.23868 31.238675 31.23858...,-29.73064 -29.730666999999997 -29.730573 -29.7...,kdwny kdwny kdwny kdwny kdwny kdwny kdwny kdwn...,kdwn kdwn kdwn kdwn kdwn kdwn kdwn kdwn kdwn k...,kdw kdw kdw kdw kdw kdw kdw kdw kdw kdw kdw kd...,CNSHK,ZADUR,113.86305800000001 22.559462,31.050079999999998 -29.868304,ws0b,ws0br,ws0,kdwn,kdwn4,kdw


In [18]:
# train mse:  256.9597223109227
# train mse:  256.2317758079061

sub_FORMAT = "%Y/%m%d  %H:%M:%S"
UTC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

def get_time(x, y):
    x = x.split(' ')[-1]
    x = datetime.datetime.strptime(x, UTC_FORMAT)
    day = y // 24 
    seconds = int((y - (y//24)*24)*3600)
    res = x + datetime.timedelta(days=day, seconds=seconds)
    str_res = res.strftime(sub_FORMAT)
    return res 

# result = pd.DataFrame({
#         'loadingOrder': test_loadingOrder, 
#         'label': test_pred,
#     })

result['timestamp'] = test_df['timestamp']
result['ETA'] = list(map(lambda x, y: get_time(x, y), result['timestamp'], result['label']))

In [19]:
result.shape 

(239, 4)

In [20]:
test_data = pd.read_csv("data/Btest0711_ALL.csv") 
def get_data(data, mode='train'):
    
    assert mode=='train' or mode=='test'
#     if mode=='train':
#         data['vesselNextportETA'] = pd.to_datetime(data['vesselNextportETA'], infer_datetime_format=True) # 转换时间
    if mode=='test':
        data['temp_timestamp'] = data['timestamp']
        data['onboardDate'] = pd.to_datetime(data['onboardDate'], infer_datetime_format=True)
    
    data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format=True)
    return data

test_data = get_data(test_data, mode='test')

test_data.sort_values(['loadingOrder', 'timestamp'], inplace=True)
test_data = test_data.reset_index(drop=True)
print(test_data.shape)

test_data = test_data.merge(result[['loadingOrder', 'ETA']], on='loadingOrder', how='left')
print(test_data.shape)

test_data['ETA'] = test_data['ETA'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data.drop(['direction','TRANSPORT_TRACE'],axis=1,inplace=True)
test_data['onboardDate'] = test_data['onboardDate'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data['creatDate'] = pd.datetime.now().strftime('%Y/%m/%d  %H:%M:%S')
test_data['timestamp'] = test_data['temp_timestamp']
# 整理columns顺序
result = test_data[['loadingOrder', 'timestamp', 'longitude', 'latitude', 'carrierName', 'vesselMMSI', 'onboardDate', 'ETA', 'creatDate']]

(57071, 11)
(57071, 12)


/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:26: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.


In [21]:
result.to_csv('result/round2B_lgb_hc.csv', index=False)

In [23]:
tmp[['loadingOrder', 'onboardDate', 'ETA']].to_csv("result/res.csv", index=None)

In [168]:
# 合并结果
res1 = pd.read_csv("result/round2_lgb_0805_lr001.csv")
res1.sort_values(['loadingOrder', 'timestamp'], inplace=True)
res1 = res1.reset_index(drop=True)
res2 = pd.read_csv("result/round2_804_newdata_more_label.csv")
res2.sort_values(['loadingOrder', 'timestamp'], inplace=True)
res2 = res2.reset_index(drop=True)
print(res1.shape, res2.shape)

res1['ETA'] = pd.to_datetime(res1['ETA'], infer_datetime_format=True)
res2['ETA'] = pd.to_datetime(res2['ETA'], infer_datetime_format=True)

res1['diff'] = (res1['ETA'] - pd.to_datetime('2019/01/01 00:00:00',infer_datetime_format=True)).dt.total_seconds()/3600
res2['diff'] = (res2['ETA'] - pd.to_datetime('2019/01/01 00:00:00',infer_datetime_format=True)).dt.total_seconds()/3600

res1['diff'] = res1['diff'] * 0.3 + res2['diff'] * 0.7

sub_FORMAT = "%Y/%m%d  %H:%M:%S"
UTC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

x = pd.to_datetime('2019/01/01 00:00:00',infer_datetime_format=True)
def get_time(y):
    day = y // 24 
    seconds = int((y - (y//24)*24)*3600)
    res = x + datetime.timedelta(days=day, seconds=seconds)
    str_res = res.strftime(sub_FORMAT)
    return res 

res1['ETA'] = res1['diff'].apply(lambda x: get_time(x))
del res1['diff']

res1['ETA'] = res1['ETA'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))

(72847, 9) (72847, 9)


In [169]:
result['ETA'] = res1['ETA']

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [174]:
result.to_csv("result/round2_lgb_ronghe0808.csv", index=False)

In [170]:
res1.drop_duplicates('loadingOrder')

,loadingOrder,timestamp,longitude,latitude,carrierName,vesselMMSI,onboardDate,ETA,creatDate
0,AC437723355280,2019-06-23T02:53:18.000Z,113.895823,22.383463,JCMFTA,G9916514058,2019/06/23 02:53:18,2019/07/20 20:51:08,2020/08/05 14:09:39
334,AR701843002140,2020-03-25T15:55:58.000Z,113.694225,22.630953,OYSCFP,O8358265987,2020/03/25 15:49:58,2020/03/29 03:25:32,2020/08/05 14:09:39
372,AT407433613767,2019-04-01T00:14:18.000Z,104.196312,1.287627,RWHZVZ,J3697709469,2019/04/01 00:14:18,2019/04/24 10:48:44,2020/08/05 14:09:39
550,AV639724236766,2020-06-01T19:13:22.000Z,113.894683,22.451200,OYSCFP,R7006912480,2020/06/01 19:13:22,2020/06/12 22:33:21,2020/08/05 14:09:39
690,BA498843975994,2020-06-16T02:23:12.000Z,113.899117,22.461683,JONOCD,U9615709922,2020/06/16 02:23:12,2020/06/19 20:28:43,2020/08/05 14:09:39
1790,BB470439135271,2020-04-28T00:00:19.000Z,129.496267,34.140347,OIEQNT,Q2654292540,2020/04/28 00:00:19,2020/05/28 08:07:55,2020/08/05 14:09:39
3867,BE790761427541,2020-06-10T13:59:56.000Z,113.883267,22.444100,OYSCFP,I9946011301,2020/06/10 13:59:56,2020/06/26 16:00:00,2020/08/05 14:09:39
4029,BE898272362291,2019-11-19T05:07:38.000Z,114.273142,22.563410,OYSCFP,C2075927370,2019/11/19 05:07:38,2019/12/12 15:37:20,2020/08/05 14:09:39
4121,BE929753510083,2019-07-06T23:04:08.000Z,114.285403,22.576393,OYSCFP,B8183659437,2019/07/06 23:04:08,2019/07/31 00:36:07,2020/08/05 14:09:39
4210,BK898009012594,2019-09-05T07:01:18.000Z,114.274107,22.564852,OIEQNT,E8197708075,2019/09/05 07:01:18,2019/09/27 05:03:11,2020/08/05 14:09:39


In [172]:
# result.drop_duplicates('loadingOrder')

In [157]:
res1 = pd.read_csv("result/round2_lgb_0805_lr001.csv")
res1.drop_duplicates('loadingOrder')

,loadingOrder,timestamp,longitude,latitude,carrierName,vesselMMSI,onboardDate,ETA,creatDate
0,AC437723355280,2019-06-23T02:53:18.000Z,113.895823,22.383463,JCMFTA,G9916514058,2019/06/23 02:53:18,2019/07/20 22:31:16,2020/08/05 14:09:39
334,AR701843002140,2020-03-25T15:55:58.000Z,113.694225,22.630953,OYSCFP,O8358265987,2020/03/25 15:49:58,2020/03/29 10:27:36,2020/08/05 14:09:39
372,AT407433613767,2019-04-01T00:14:18.000Z,104.196312,1.287627,RWHZVZ,J3697709469,2019/04/01 00:14:18,2019/04/23 23:46:27,2020/08/05 14:09:39
550,AV639724236766,2020-06-01T19:13:22.000Z,113.894683,22.451200,OYSCFP,R7006912480,2020/06/01 19:13:22,2020/06/12 20:27:58,2020/08/05 14:09:39
690,BA498843975994,2020-06-16T02:23:12.000Z,113.899117,22.461683,JONOCD,U9615709922,2020/06/16 02:23:12,2020/06/19 17:06:21,2020/08/05 14:09:39
1790,BB470439135271,2020-04-28T00:00:19.000Z,129.496267,34.140347,OIEQNT,Q2654292540,2020/04/28 00:00:19,2020/05/28 09:14:58,2020/08/05 14:09:39
3867,BE790761427541,2020-06-10T13:59:56.000Z,113.883267,22.444100,OYSCFP,I9946011301,2020/06/10 13:59:56,2020/06/26 16:20:44,2020/08/05 14:09:39
4029,BE898272362291,2019-11-19T05:07:38.000Z,114.273142,22.563410,OYSCFP,C2075927370,2019/11/19 05:07:38,2019/12/12 18:48:47,2020/08/05 14:09:39
4121,BE929753510083,2019-07-06T23:04:08.000Z,114.285403,22.576393,OYSCFP,B8183659437,2019/07/06 23:04:08,2019/07/30 13:05:19,2020/08/05 14:09:39
4210,BK898009012594,2019-09-05T07:01:18.000Z,114.274107,22.564852,OIEQNT,E8197708075,2019/09/05 07:01:18,2019/09/26 02:54:08,2020/08/05 14:09:39


In [140]:
res2.head()

,loadingOrder,timestamp,longitude,latitude,carrierName,vesselMMSI,onboardDate,ETA,creatDate,diff
0,AC437723355280,2019-06-23T02:53:18.000Z,113.895823,22.383463,JCMFTA,G9916514058,2019/06/23 02:53:18,2019-07-20 20:08:14,2020/08/04 21:29:26,4820.137222
1,AC437723355280,2019-06-23T03:01:28.000Z,113.917878,22.360128,JCMFTA,G9916514058,2019/06/23 02:53:18,2019-07-20 20:08:14,2020/08/04 21:29:26,4820.137222
2,AC437723355280,2019-06-23T03:16:38.000Z,113.977712,22.345253,JCMFTA,G9916514058,2019/06/23 02:53:18,2019-07-20 20:08:14,2020/08/04 21:29:26,4820.137222
3,AC437723355280,2019-06-23T03:17:08.000Z,113.980273,22.345310,JCMFTA,G9916514058,2019/06/23 02:53:18,2019-07-20 20:08:14,2020/08/04 21:29:26,4820.137222
4,AC437723355280,2019-06-23T03:18:08.000Z,113.984088,22.345340,JCMFTA,G9916514058,2019/06/23 02:53:18,2019-07-20 20:08:14,2020/08/04 21:29:26,4820.137222


# 分析一下结果

In [6]:
import pandas as pd
# res1 = pd.read_csv("data/B_lgb_10fold_old_add_fea.csv")
res2 = pd.read_csv("data/B_lgb_10fold_old.csv")
res1.drop_duplicates('loadingOrder', inplace=True)
res2.drop_duplicates('loadingOrder', inplace=True)

res1['onboardDate'] = pd.to_datetime(res1['onboardDate'], infer_datetime_format=True)
res1['ETA'] =  pd.to_datetime(res1['ETA'], infer_datetime_format=True)
res2['onboardDate'] = pd.to_datetime(res2['onboardDate'], infer_datetime_format=True)
res2['ETA'] =  pd.to_datetime(res2['ETA'], infer_datetime_format=True)

res1['diff'] = (res1['ETA'] - res1['onboardDate']).dt.total_seconds()
res2['diff'] = (res2['ETA'] - res2['onboardDate']).dt.total_seconds()

In [7]:
res1['diff'].corr(res2['diff'])

0.9994296946716894

In [29]:
result.shape

(57071, 9)

In [65]:
res1 = pd.read_csv("result/round2B_lgb_hc.csv")
# res1 = result
# res2 = pd.read_csv("result/nn_merge3.csv")
res2 = pd.read_csv("result/b2_807.csv")

In [66]:
res1 = res1.drop_duplicates("loadingOrder")
res2 = res2.drop_duplicates("loadingOrder")

In [67]:
res1 = res1[['loadingOrder', 'timestamp', 'ETA']].reset_index(drop=True)
res2 = res2[['loadingOrder', 'timestamp', 'ETA']].reset_index(drop=True)

In [68]:
res1 = res1.merge(res2, how='left', on='loadingOrder')

In [69]:
del res1['timestamp_y']

In [70]:
res2

,loadingOrder,timestamp,ETA
0,NJ169522947117,2019-12-30T06:23:18.000Z,2020/01/01 19:19:59
1,SX540070026140,2020-03-11T05:59:18.000Z,2020/03/13 20:19:54
2,ZV919459607351,2020-06-01T00:09:13.000Z,2020/06/07 09:42:46
3,AE378244933121,2020-06-01T00:06:13.000Z,2020/06/08 05:09:05
4,FQ684650477699,2020-06-01T00:00:43.000Z,2020/06/25 12:23:58
5,WG447072368140,2020-06-01T00:00:12.000Z,2020/06/18 07:32:15
6,ZB832047068480,2020-06-01T00:10:23.000Z,2020/06/02 19:07:00
7,JU877708983002,2020-06-01T00:07:12.000Z,2020/06/09 20:52:12
8,TI932873678567,2020-06-08T08:12:02.000Z,2020/06/18 20:51:11
9,KX661536528233,2020-06-01T00:05:39.000Z,2020/06/22 07:35:53


In [71]:
res1['ETA_x'] = pd.to_datetime(res1['ETA_x'], infer_datetime_format=True)
res1['ETA_y'] = pd.to_datetime(res1['ETA_y'], infer_datetime_format=True)

In [72]:
count = test_data.groupby('loadingOrder').agg({'timestamp':'count'}).reset_index()
res1 = res1.merge(count, how='left', on='loadingOrder')

res1['diff'] = ((res1['ETA_x'] - res1['ETA_y']).dt.total_seconds()/3600) ** 2 * res1['timestamp'] / 57071

In [73]:
res1['diff'] = round(res1['diff'], 3)

In [74]:
res2.sort_values(['loadingOrder', 'timestamp'], inplace=True)
res2

,loadingOrder,timestamp,ETA
32,AB674675500650,2020-06-01T00:01:50.000Z,2020/06/09 03:41:28
59,AC710522369160,2020-06-09T23:17:33.000Z,2020/06/15 13:44:48
110,AD584528667006,2020-06-02T03:33:34.000Z,2020/06/04 16:50:39
76,AD852352572660,2020-06-10T18:56:34.000Z,2020/06/24 09:08:29
3,AE378244933121,2020-06-01T00:06:13.000Z,2020/06/08 05:09:05
54,AE563102255135,2019-12-26T00:24:48.000Z,2020/01/31 09:36:18
179,AI823168582586,2019-08-27T02:51:28.000Z,2019/09/20 09:30:58
155,AJ216944869611,2019-05-13T03:00:08.000Z,2019/06/06 13:02:37
234,AM438554857021,2019-06-24T08:17:38.000Z,2019/07/17 07:34:21
95,AO528516733940,2020-03-22T12:30:18.000Z,2020/03/26 13:09:48


In [76]:
res1[res1['diff'] > 5]

,loadingOrder,timestamp_x,ETA_x,ETA_y,timestamp,diff
0,AB674675500650,2020-06-01T00:01:50.000Z,2020-06-07 05:44:09,2020-06-09 03:41:28,440,16.282
5,AE563102255135,2019-12-26T00:24:48.000Z,2020-01-30 08:01:53,2020-01-31 09:36:18,934,10.703
28,CM821215596820,2020-02-07T08:01:58.000Z,2020-03-13 23:33:52,2020-03-15 08:45:37,280,5.406
58,FX975491217543,2020-06-01T00:01:50.000Z,2020-06-10 11:29:01,2020-06-13 07:21:33,440,35.519
105,KO849768330117,2020-06-04T15:56:36.000Z,2020-06-09 22:34:02,2020-06-13 18:29:15,88,13.028
133,NX668481428339,2020-06-01T00:07:20.000Z,2020-06-25 04:22:06,2020-06-28 09:46:44,175,18.375
175,SF782540171392,2020-01-14T02:21:58.000Z,2020-02-07 21:19:44,2020-02-07 05:12:21,1468,6.687
178,SU860129068725,2019-09-01T01:09:08.000Z,2019-09-21 18:39:11,2019-09-24 13:28:32,99,7.746
190,UB681657714929,2020-06-03T07:08:36.000Z,2020-06-25 06:04:51,2020-06-27 03:13:07,145,5.176
200,UU854609945095,2019-02-07T00:24:28.000Z,2019-03-03 10:00:16,2019-02-28 21:54:15,936,59.240


In [39]:
test_data[test_data['loadingOrder'] == 'OS590961425176'].shape

(190, 11)

In [51]:
res1['diff'].sum()

1061.188

In [62]:
def cal(hour, count):
    return hour**2 * count / 72847

cal(3.6, 334) * 72847

4328.64